In [21]:
import os
import json
import openai
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
def clean_json_tags(json_str):
    return json_str.replace("```json", "").replace("```", "").strip()

In [7]:
user_input_list = ["你好，今天天气如何", "我想问一下我买的裤子的一些问题", "我要退货", "我上周买的裤子可能不适合我"]
user_intent_list = ["用户要求退货", "用户需求导购"]
glossary = ["导购:引导用户发生购买行为"]

In [26]:
llm_openai = openai.OpenAI(api_key=OPENAI_API_KEY)
# llm_deepseek = openai.OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")
system_prompt = "你是一个识别用户意图的有用的助手"
glossary_in_prompt = "\n".join(glossary)

user_input = user_input_list[2]
user_intent = user_intent_list[0]

prompt = f"""用户的话: {user_input}
意图：{user_intent}
用户的话满足所设定的意图么?以下列json的格式输出结果：
```json
{{
    "user_input": "user_input",
    "user_intent": "user_intent",
    "is_match": "True/False"
}}
```
不明白的术语可以参考下列术语表：
术语：{glossary_in_prompt}
"""
response = llm_openai.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            stream=False
        )

# response = llm_deepseek.chat.completions.create(
#             model="deepseek-chat",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": prompt}
#             ],
#             stream=False
#         )

response_content = clean_json_tags(response.choices[0].message.content)


In [27]:
output = json.loads(response_content)

In [31]:
if(output["is_match"].lower() == "true"):
    print(f"用户的话: {user_input} 意图：{user_intent} 满足")
else:
    print(f"用户的话: {user_input} 意图：{user_intent} 不满足")


用户的话: 我要退货 意图：用户要求退货 满足
